In [62]:
import numpy as np
import pandas as pd
import string
import re
from fuzzywuzzy import fuzz

<h2>Reading in the datasets</h2>

In [2]:
dna = pd.read_csv("../data/working/validcompaniesdictionary.csv", index_col = [0])

In [3]:
fda = pd.read_excel("../data/original/fda_companies.xlsx")

In [471]:
ndc = pd.read_excel("../data/original/BI DSPG Company Datasets/NDC_Company_Dataset.xls")

<h2>Neil's code for cleaning</h2>

In [5]:
removeset=string.punctuation
removeset=removeset.replace("-","") #Don't remove dashes
removeset=removeset.replace("&","") #Don't remove ampersand
removeset=removeset.replace("_","") #Don't remove underscore
removeset=removeset.replace("%","") #Don't remove percent
removeset=removeset.replace("$","") #Don't remove dollar
   
print(removeset)

!"#'()*+,./:;<=>?@[\]^`{|}~


In [183]:
# remove all single characters (This step is done first, because later there are single chars we want to retain.)
#document = re.sub(r'\s+[a-zA-Z]\s+', ' ', str(X[sen]))
string = "hello i world"
string = re.sub(r'\s+[a-zA-Z]\s+', ' ', string)
print(string)

# remove all numbers
#document = re.sub(r'[0-9]','', document)
string2 = "h3ll0"
string2 = re.sub(r'[0-9]','', string)
print(string2)

# Substituting multiple spaces with single space
#document = re.sub(r'\s+', ' ', document, flags=re.I)
string3 = "hello      world"
string3 = re.sub(r'\s+', ' ', string3, flags=re.I)
print(string3)

#Converting to lowercase
string4 = "HEllo WORLD"
string4 = string4.lower()
print(string4)

#Removing prefixed 'b'
#document = re.sub(r'^b\s+', '', document)
string5 = "b hello world"
string5 = re.sub(r'^b\s+', '', string5)
print(string5)

#Make dashes into combined words
#document = re.sub(r'\s-\s+', '-', document)
string6 = "hello - world"
string6 = re.sub(r'\s-\s+', '-', string6)
print(string6)

#Make ampersand into combined words
#document = re.sub(r'\s&\s+', '&', document)
string7 = "hel & lo & world"
string7 = re.sub(r'\s&\s+', '&', string7)
print(string7)

#Make underscore into combined words
#document = re.sub(r'\s_\s+', '_', document)
string8 = "hel _ lo wo _ rld"
string8 = re.sub(r'\s_\s+', '_', string8)
print(string8)

hello world
hello world
hello world
hello world
hello world
hello-world
hel&lo&world
hel_lo wo_rld


In [77]:
#removes all punctuation in string that is in removeset
document = "Johnson+;Johnson!"
for i in removeset:
    document=re.sub(re.escape(i),"",document)
print(document)

JohnsonJohnson


<h2>Cleaning NDC</h2>

<h4>Removing the first 25 since they are just numbers</h4>

In [472]:
#Getting rid of the first 25 since those are just numbers
ndc = ndc.iloc[25:]
#renaming column
ndc = ndc.rename(columns = {'Row Labels':'company'})

<h4>Lowercase everything</h4>

In [473]:
#Converting to lower first
ndc.company = ndc.company.str.lower()

<h4>Remove content that are in parentheses</h4>

In [474]:
#Function for removing parentheses content
def removeParenthesesContent(string):
    return re.sub(r'\([^)]*\)', '', string)


In [475]:
ndc['companiesWithoutParensContent'] = ndc['company'].apply(removeParenthesesContent)

In [476]:
del ndc['company']

In [477]:
ndc.head()

,companiesWithoutParensContent
25,spironolactone 2%
26,-l'oreal usa products inc
27,.cardinal health
28,.church & dwight canada corp
29,{preferred pharmaeutials inc.


<h4>Remove Unwanted Punctuation</h4>

In [478]:
"""
Ignore 
#Function that uses regex to remove parentheses and square brackets
def removeParenthesis(string):
    return re.sub('[()\{}]', '', string)
"""
#function that gets rid of unwanted punctuation
#This does get rid of ' within a string (ex. l'oreal becomes l oreal) so maybe recheck?
def removeUnwantedPunc(string):
    return re.sub('[!"#\'()*+,./:;<=>?@[\]^`{|}~]', '', string)

In [479]:
ndc['companiesWithoutUnwantedPunc'] = ndc['companiesWithoutParensContent'].apply(removeUnwantedPunc)

In [480]:
del ndc['companiesWithoutParensContent']

In [481]:
ndc.rename(columns = {"companiesWithoutUnwantedPunc": "company"}, inplace = True)

In [482]:
#At this point all the companies are lowercased, don't have parenthetical content, and don't have unwanted punctuation
ndc.head()

,company
25,spironolactone 2%
26,-loreal usa products inc
27,cardinal health
28,church & dwight canada corp
29,preferred pharmaeutials inc


<h4>Removing numbers</h4>

In [483]:
#Function that will remove numbers
def removeNumbers(string):
    return re.sub(r'[0-9+]','', string)

In [484]:
ndc['companiesWithNoNumbers'] = ndc.company.apply(removeNumbers)

In [485]:
del ndc['company']

In [486]:
ndc = ndc.rename(columns = {"companiesWithNoNumbers": 'company'})

In [487]:
ndc.head()

,company
25,spironolactone %
26,-loreal usa products inc
27,cardinal health
28,church & dwight canada corp
29,preferred pharmaeutials inc


<h4>Removing Single Chars</h4>

In [488]:
def removeSingleChar(string):
    return re.sub(r'\s+[a-zA-Z]\s+', ' ', string)

In [489]:
ndc['companiesWithNoSingleChar'] = ndc.company.apply(removeSingleChar)

In [490]:
del ndc['company']
ndc = ndc.rename(columns={"companiesWithNoSingleChar": 'company'})

<h4>Substituting Multiple Spaces with Single Space</h4>

In [491]:
def subMultipleSpacesForOne(string):
    return re.sub(r'\s+', ' ', string, flags=re.I)

In [492]:
ndc['noMultipleSpaces'] = ndc.company.apply(subMultipleSpacesForOne)

In [493]:
del ndc['company']
ndc = ndc.rename(columns = {"noMultipleSpaces": 'company'})

<h4>Removing Prefix b</h4>

In [494]:
def removePrefix(string):
    return re.sub(r'^b\s+', '', string)

In [495]:
ndc['removedPrefix'] = ndc.company.apply(removePrefix)

In [496]:
ndc.columns

Index(['company', 'removedPrefix'], dtype='object')

In [497]:
del ndc['company']
ndc = ndc.rename(columns = {"removedPrefix":"company"})

<h4>Make dashes into combined words</h4>

In [498]:
#start here
def makeDashCombined(string):
    return re.sub(r'\s-\s+', '-', string)

In [499]:
ndc['combinedDash'] = ndc.company.apply(makeDashCombined)

In [500]:
del ndc['company']
ndc = ndc.rename(columns = {"combinedDash": 'company'})

<h4>Combine Ampersand</h4>

In [501]:
def combineAmpersand(string):
    return re.sub(r'\s&\s+', '&', string)

In [502]:
ndc['combineAmp'] = ndc.company.apply(combineAmpersand)

In [503]:
ndc.columns

Index(['company', 'combineAmp'], dtype='object')

In [504]:
del ndc['company']
ndc = ndc.rename(columns = {"combineAmp": 'company'})

In [505]:
ndc.columns

Index(['company'], dtype='object')

<h4>Combine '_'</h4>

In [506]:
def combinedUnderScore(string):
    return re.sub(r'\s_\s+', '_', string)

In [507]:
ndc.columns

Index(['company'], dtype='object')

In [508]:
ndc['combinedUnder'] = ndc.company.apply(combinedUnderScore)

In [509]:
del ndc['company']
ndc = ndc.rename(columns = {"combinedUnder": 'company'})

<h2>End Initial Cleaning</h2>

<h4>Grabbing the list of legal entities from os github</h4>

In [510]:
legalEntities = pd.read_csv("https://raw.githubusercontent.com/DSPG-Young-Scholars-Program/dspg20oss/danBranch/ossPy/keyFiles/curatedLegalEntitesRaw.csv", quotechar = "'",header = None)
legalEntities.head()

,0
0,(?i) Inc\b
1,(?i) Ltd\b
2,(?i) LLC\b
3,(?i) GmbH\b
4,(?i) Corporation\b


<h4>Using Daniel's code to get remove legal entities</h4>

In [511]:
def eraseFromColumn(inputColumn, eraseList):
    "iteratively delete regex query matches from input list"
    
    """
    inputColumn -- a column from a pandas dataframe, this will be the set of
    target words/entries that deletions will be made from
    eraseList -- a column containing strings (regex expressions) which will be
    deleted from the inputColumn, in an iterative fashion
    """
    eraseList['changeNum'] = 0
    eraseList['changeIndexes'] = ''
    
    inputColumn = inputColumn.replace(regex=True, to_replace = "\\\\", value='/')
    
    for index, row in eraseList.iterrows():
        curReplaceVal = row[0]
        currentRegexExpression=re.compile(curReplaceVal)
        CurrentBoolVec=inputColumn.str.contains(currentRegexExpression, na= False)
        eraseList['changeIndexes'].iloc[index]=[i for i, x in enumerate(CurrentBoolVec) if x]
        eraseList['changeNum'].iloc[index] = len(eraseList['changeIndexes'].iloc[index])
        inputColumn.replace(regex=True, to_replace=currentRegexExpression,value='', inplace = True)
    
    return inputColumn, eraseList

In [512]:
output, eraseList = eraseFromColumn(ndc.company, legalEntities)

In [513]:
#This dataframe is lowercased, doesn't have any of the unwanted punctuation, and has removed all legal entity labels
output= pd.DataFrame(output)

<h4>Counting Unique String Tokens using Daniel's code, then adding more to the list from Isabel and Susweta's lists</h4>

In [514]:
ndc_cleaned = output

In [515]:
longString = ndc_cleaned['company'].str.cat(sep = " ")

In [516]:
longStringSeperated = longString.split(' ')

In [517]:
uniqueSubTokenFrame = pd.DataFrame(longStringSeperated)

In [518]:
columnUniqueCounts = uniqueSubTokenFrame.iloc[:,0].value_counts()

In [519]:
tableUniqueCounts = columnUniqueCounts.reset_index()

In [520]:
tableUniqueCounts.rename(columns = {0: "count", "index":"token"}, inplace = True)

In [521]:
top20 = tableUniqueCounts.head(20).token.tolist()

In [522]:
top20

['pharmaceuticals',
 'medical',
 'products',
 'laboratories',
 'pharma',
 'anda',
 'supply',
 'health',
 'pharmaceutical',
 'usa',
 'international',
 'care',
 'and',
 'nda',
 'coltd',
 'the',
 'home',
 'healthcare',
 '',
 'of']

In [523]:
top20.append("group")
top20.append("holdings")
top20.append("capital")
top20.append("technologies")
top20.append("association")
top20.append('us')
top20.append('services')
top20.append("university")
top20.append("bank")
top20.append("partners")
top20.append("energy")
top20.append("systems")
top20.append("intl")
top20.append("pharms")
top20.append("american")
top20.append("national")

In [524]:
top20

['pharmaceuticals',
 'medical',
 'products',
 'laboratories',
 'pharma',
 'anda',
 'supply',
 'health',
 'pharmaceutical',
 'usa',
 'international',
 'care',
 'and',
 'nda',
 'coltd',
 'the',
 'home',
 'healthcare',
 '',
 'of',
 'group',
 'holdings',
 'capital',
 'technologies',
 'association',
 'us',
 'services',
 'university',
 'bank',
 'partners',
 'energy',
 'systems',
 'intl',
 'pharms',
 'american',
 'national']

<h4>Getting rid of top occuring tokens</h4>

In [525]:
ndc_cleaned = ndc_cleaned['company'].apply(lambda x: ' '.join([word for word in x.split() if word not in (top20)])) #Isabel's code

In [526]:
ndc_cleaned = pd.DataFrame(ndc_cleaned)

In [527]:
ndc_cleaned

,company
25,spironolactone %
26,-loreal
27,cardinal
28,church&dwight canada
29,preferred pharmaeutials
...,...
7046,zygone
7047,zyla life sciences
7048,name
7049,


<h4>Adding csv to working data</h4>

In [528]:
ndc_cleand.to_csv("../data/working/ndc_clean.csv")